<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: COVID-19 Dashboard
Caderno de **Aula**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Análise Exploratória de Dados;</li>
  <li>Visualização Interativa de Dados;</li>
  <li>Storytelling.</li>
</ol>

---

# **Aulas**

## 1\. Introdução

### **1.1. TLDR**

 - **Dashboard**:
  - Google Data Studio ([link](https://lookerstudio.google.com/reporting/08819073-431e-4f2a-b594-16c02177d8a8)).
 - **Processamento**:
  - Kaggle Notebook ([link](https://www.kaggle.com/fabiopereira445/projeto-ebac-covid19)).
 - **Fontes**:
  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

### **1.2. Pandemia Coronavírus 2019**

> A COVID-19 é uma infecção respiratória aguda causada pelo coronavírus SARS-CoV-2, potencialmente grave, de elevada transmissibilidade e de distribuição global. Fonte: Governo brasileiro ([link](https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus)).

A disponibilidade de dados sobre a evolução da pandemia no tempo em uma determinada região geográfica é fundamental para o seu combate! Este projeto busca construir um dashboard de dados para exploração e visualização interativa de dados sobre o avanço de casos e da vacinação do Brasil. O processamento de dados está neste `https://www.kaggle.com/fabiopereira445/projeto-ebac-covid19` e o dashboard, neste `https://lookerstudio.google.com/reporting/08819073-431e-4f2a-b594-16c02177d8a8`.

### **1.3. Dados**

Os dados sobre **casos da COVID-19** são compilados pelo centro de ciência de sistemas e engenharia da universidade americana **John Hopkins** ([link](https://www.jhu.edu)). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.). O website do projeto pode ser acessado neste [link](https://systems.jhu.edu/research/public-health/ncov/) enquanto os dados, neste [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país;
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_1d**: número diário de infectados;
 - **confirmed_moving_avg_7d**: média móvel de 7 dias do número diário de infectados;
 - **confirmed_moving_avg_7d_rate_14d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortos;
 - **deaths_1d**: número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias do número diário de mortos;
 - **deaths_moving_avg_7d**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **month**: mês de referência;
 - **year**: ano de referência.

Os dados sobre **vacinação da COVID-19** são compilados pelo projeto Nosso Mundo em Dados (*Our World in Data* ou OWID) da universidade britânica de **Oxford** ([link](https://www.ox.ac.uk)). Os dados são **atualizados diariamente** deste janeiro de 2020 com uma **granularidade temporal de dias e geográfica de países**. O website do projeto pode ser acessado neste [link](https://ourworldindata.org) enquanto os dados, neste [link](https://covid.ourworldindata.org/data/owid-covid-data.csv). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **country**: país;
 - **population**: população estimada;
 - **total**: número acumulado de doses administradas;
 - **one_shot**: número acumulado de pessoas com uma dose;
 - **one_shot_perc**: número acumulado relativo de pessoas com uma dose;
 - **two_shots**: número acumulado de pessoas com duas doses;
 - **two_shot_perc**: número acumulado relativo de pessoas com duas doses;
 - **three_shots**: número acumulado de pessoas com três doses;
 - **three_shot_perc**: número acumulado relativo de pessoas com três doses;
 - **month**: mês de referência;
 - **year**: ano de referência.

## 2\. Análise Exploratória de Dados

Nesta sessão vamos utilizar os seguintes pacotes Python para processar os dados bruto em um formato adequado para um painel para exploração interativa de dados.

In [ ]:
# Importando as bibliotecas necessárias
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

### **2.1. Casos**

Vamos processar os dados de **casos** da universidade John Hopkins.



#### **2.1.1. Extração**

O dado está compilado em um arquivo por dia, exemplo para 2021/12/01.

In [ ]:
# Carregando os dados de vacinação
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [ ]:
# Exibindo as primeiras linhas do dataframe
cases.head()

Portanto, precisaremos iterar dentro de um intervalo de tempo definido para extraí-lo.

In [ ]:
# Função para gerar um intervalo de datas
#Vamos filtrar a base de dados de acordo com a coluna date para garantir que ambas as bases de dados tratem do mesmo período de tempo.
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
  date_range_days: int = (end_date - start_date).days
  for lag in range(date_range_days):
    yield start_date + timedelta(lag)

In [ ]:
# Definindo as datas de início e fim para a extração dos dados
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

De maneira iterativa, vamos selecionar as colunas de interesse e as linhas referentes ao Brasil.

---
Observação:
na aula, o professor utiliza o método `append()`, mas esse método foi removido da
biblioteca Pandas (foi descontinuado desde a versão 1.4 e removido a partir da versão 2.0). [Link da documentação para mais informações](https://pandas.pydata.org/pandas-docs/version/1.5/whatsnew/v1.4.0.html#whatsnew-140-deprecations-frame-series-append)

Seguindo a sintaxe recomendada pela documentação, segue abaixo o código atualizado utilizando `concat()`. [Link da documentação: pandas.concat()](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)

---



In [ ]:
# Inicializando as variáveis para armazenar os dados
cases = None
cases_is_empty = True
# Iterando sobre o intervalo de datas
for date in date_range(start_date=start_date, end_date=end_date):

  date_str = date.strftime('%m-%d-%Y')
  data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

  case = pd.read_csv(data_source_url, sep=',')

  case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
  case = case.query('Country_Region == "Brazil"').reset_index(drop=True)
  case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))
  if cases_is_empty:
    cases = case
    cases_is_empty = False
  else:
    cases = pd.concat([cases, case], axis= 0, ignore_index=True)

In [ ]:
# Exibindo as primeiras linhas dos dados filtrados para o estado de São Paulo
cases.query('Province_State == "Sao Paulo"').head()

#### **2.1.2. Wrangling**

Vamos manipular os dados para o dashboard. O foco é em garantir uma boa granularidade e qualidade da base de dados.

In [ ]:
# Exibindo as primeiras linhas do dataframe
cases.head()

In [ ]:
# Exibindo a forma (número de linhas e colunas) do dataframe
cases.shape

In [ ]:
# Exibindo informações gerais sobre o dataframe
cases.info()

Começamos com o nome das colunas.

In [ ]:
# Renomeando as colunas para facilitar a manipulação
cases = cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)
# Convertendo todos os nomes das colunas para minúsculas
for col in cases.columns:
  cases = cases.rename(columns={col: col.lower()})

Ajustamos o nome dos estados.

In [ ]:
# Ajustando os nomes dos estados para o formato correto
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

Vamos então computar novas colunas para enriquecer a base de dados.

 - Chaves temporais:

In [ ]:
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

 - População estimada do estado:

In [ ]:
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))
cases = cases.drop('incident_rate', axis=1)

 - Número, média móvel (7 dias) e estabilidade (14 dias) de casos e mortes por estado:

| 1 | 2 | 3 | 4 | 5 | 6 | <font color='red'>7</font> | <font color='green'>8</font> | 9 | 10 | 11 | 12 | 13 | <font color='blue'>14<font color='red'> | 15 | 16 | 17 | 18 | 19 | 20 | 21 |
| - | - | - | - | - | - | - | - | - | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |
| <font color='red'>D-6</font> | <font color='red'>D-5</font> | <font color='red'>D-4</font> | <font color='red'>D-3</font> | <font color='red'>D-2</font> | <font color='red'>D-1</font> | <font color='red'>D0</font> | | | | | | | | | | | | | | |
| D-7 | <font color='green'>D-6</font> | <font color='green'>D-5</font> | <font color='green'>D-4</font> | <font color='green'>D-3</font> | <font color='green'>D-2</font> | <font color='green'>D-1</font> | <font color='green'>D0</font> | | | | | | | | | | | | | |
| D-13 | D-12 | D-11 | D-10 | D-9 | D-8 | D-7 | <font color='blue'>D-6</font> | <font color='blue'>D-5</font> | <font color='blue'>D-4</font> | <font color='blue'>D-3</font> | <font color='blue'>D-2</font> | <font color='blue'>D-1</font> | <font color='blue'>D0</font> | | | | | | | |



---
Observação: como explicado anteriormente, o código revisado utilizando `concat()` está apresentado abaixo.


---




In [ ]:
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:

  if np.isnan(rate):
    return np.NaN

  if rate < 0.85:    # Se a divisão for menor que 0.85
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status


for state in cases['state'].drop_duplicates():

  cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
  cases_per_state = cases_per_state.sort_values(by=['date'])

  cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
  cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
  cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
  cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

  cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
  cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
  cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
  cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

  if cases_is_empty:
    cases_ = cases_per_state
    cases_is_empty = False
  else:
    cases_ = pd.concat([cases_, cases_per_state],axis=0, ignore_index=True)

cases = cases_
cases_ = None

Garantir o tipo do dado é fundamental para consistência da base de dados. Vamos fazer o *type casting* das colunas.

In [ ]:
cases['population'] = cases['population'].astype('Int64')
cases['confirmed_1d'] = cases['confirmed_1d'].astype('Int64')
cases['confirmed_moving_avg_7d'] = cases['confirmed_moving_avg_7d'].astype('Int64')
cases['deaths_1d'] = cases['deaths_1d'].astype('Int64')
cases['deaths_moving_avg_7d'] = cases['deaths_moving_avg_7d'].astype('Int64')

Por fim, vamos reorganizar as colunas e conferir o resultado final.

In [ ]:
cases = cases[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

In [ ]:
cases.head(n=15)

#### **2.1.3. Carregamento**

Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [ ]:
# Salvando os dados de casos manipulados em um arquivo CSV
cases.to_csv('./covid-cases.csv', sep=',', index=False)

### **2.2. Vacinação**

Vamos processar os dados de **vacinação** da universidade de Oxford.

#### **2.2.1. Extração**

Os dados estão compilados em um único arquivo.

In [ ]:
# Carregando os dados de vacinação
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)

In [ ]:
# Visualizando dados
vaccines.head()

Vamos selecionar as colunas de interesse e as linhas referentes ao Brasil.

In [ ]:
#selecionando as colunas de interesse e as linhas referentes ao Brasil.
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

In [ ]:
# Visualizando dados selecionados
vaccines.head()

#### **2.2.2. Wrangling**

Vamos manipular os dados para o dashboard. O foco é em garantir uma boa granularidade e qualidade da base de dados.

In [ ]:
# Exibindo as primeiras linhas do dataframe filtrado
vaccines.head()

In [ ]:
# Exibindo a forma (número de linhas e colunas) do dataframe
vaccines.shape

In [ ]:
# Exibindo informações gerais sobre o dataframe
vaccines.info()

Vamos começar tratando os dados faltantes, a estratégia será a de preencher os buracos com o valor anterior válido mais próximo.

In [ ]:
# Preenchendo os valores faltantes com o valor anterior válido mais próximo
vaccines = vaccines.fillna(method='ffill')

Vamos também filtrar a base de dados de acordo com a coluna `date` para garantir que ambas as bases de dados tratam do mesmo período de tempo.

In [ ]:
# Filtrando os dados para o período de 2021
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

Agora, vamos alterar o nome das colunas.

In [ ]:
# Renomeando as colunas para facilitar a manipulação
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

Vamos então computar novas colunas para enriquecer a base de dados.

 - Chaves temporais:

In [ ]:
# Adicionando colunas de chaves temporais para análise de tendências ao longo do tempo,
# ajudando a identificar padrões sazonais e a suavizar flutuações diárias para uma visão mais clara das tendências.
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

 - Dados relativos:

In [ ]:
# Calculando dados relativos para comparar diferentes conjuntos de dados em uma base comum
#ajudando a entender a cobertura vacinal em relação à população total, permitindo comparações entre diferentes regiões ou períodos de tempo.
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

Garantir o tipo do dado é fundamental para consistência da base de dados. Vamos fazer o *type casting* das colunas.

In [ ]:
# Garantindo o tipo correto dos dados para consistência
vaccines['population'] = vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

Por fim, vamos reorganizar as colunas e conferir o resultado final.

In [ ]:
# Reorganizando as colunas e conferindo o resultado final
vaccines = vaccines[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

In [ ]:
# Exibindo as últimas linhas do dataframe final
vaccines.tail()

#### **2.2.3. Carregamento**

Com os dados manipulados, vamos persisti-lo em disco, fazer o seu download e carrega-lo no Google Data Studio.

In [ ]:
# Salvando os dados de vacinação manipulados em um arquivo CSV
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)

## 3\. Exploração Interativa de Dados

### **3.1. KPIs**

O dashboard de dados contem os seguintes indicadores chaves de desempenho (*key performance indicator* ou KPI) consolidados:

1. Casos e mortes nas 24 horas;
1. Média móvel (7 dias) de casos e mortes;
1. Tendência de casos e mortes;
1. Proporção de vacinados com 1ª, 2ª e 3ª doses.

### **3.2. EDA**

O dashboard de dados contem os seguintes gráficos para a análise exploratória de dados (*exploratory data analysis*
ou EDA) interativa:

1. Distribuição do números de casos e mortes ao longo do tempo;
1. Distribuição da média móvel (7 dias) do números de casos e mortes ao longo do tempo;
1. Distribuição geográfica dos casos por estado por dia.

## 4\. Storytelling

**Análise dos Dados de COVID-19 no Brasil - 30 de dezembro de 2021**

Caros stakeholders,

Gostaria de compartilhar uma análise detalhada do estado da pandemia de COVID-19 no Brasil, baseada nos dados atualizados até 30 de dezembro de 2021. Nosso objetivo é fornecer insights críticos que ajudarão na tomada de decisões estratégicas. Vejam abaixo os principais pontos da nossa análise:

**1. Vacinação no Brasil**

1ª Dose: 77,2% de pessoas vacinadas.

2ª Dose: 66,6% de pessoas vacinadas.

3ª Dose: Apenas 12,3% de pessoas vacinadas.

**Análise:**

 A disparidade entre as doses administradas mostra a necessidade urgente de intensificar os esforços de vacinação, especialmente para a 3ª dose.A sugestão seria estabelecer uma meta cujo objetivo seja de aumentar rapidamente o número de pessoas totalmente vacinadas, incluindo a dose de reforço, para garantir uma proteção duradoura.

**2. Casos e Mortes**
Casos nas Últimas 24h: 14,6 milhões.

Média Móvel de 7 Dias de Casos Confirmados: 14,5 milhões.

Mortes nas Últimas 24h: 423.708.

Média Móvel de 7 Dias de Mortes: 425.553.

**Análise:**

 Os números no período analisado mostram uma tendência de alta tanto nos casos quanto nas mortes, o que é preocupante.A sugestão seria tomar medidas mais rigorosas de controle de infecções, sendo imperativas para reverter essa tendência.

**3. Tendências**

Tendência de Casos: Em junho apresentamos um pico de alta em casos dentro de 24h ultrapassando a marca de 100 MIL Aumento (UPWARD).

Tendência de Mortes: Entre final de Marçoe Abril registramos um pico de mortes em 24h  chegando a máxima de 4148  (UPWARD).

**Análise:**

 A tendência de aumento contínuo reforça a necessidade de manter e intensificar as medidas de prevenção e controle para evitar a sobrecarga do sistema de saúde e reduzir a mortalidade.

**4. Distribuição Geográfica dos Casos**

O mapa do Brasil incluído no dashboard mostra a distribuição dos casos por estado nas últimas 24h, com uma legenda colorida indicando o número de casos em cada estado.

**Exemplo:**

**Análise:**

 Embora o número absoluto de novos casos em São Paulo (19.467) pareça alto, a média móvel de 7 dias de 13,4 mil indica uma tendência de queda. Isso sugere que, apesar do número alto, as novas infecções estão diminuindo gradualmente, o que é um sinal positivo.

**Mortes e Tendências**

**Mortes nas Últimas 24 Horas:** 599

**Média Móvel de 7 Dias de Mortes:** 507

**Tendência de Mortes:** STABLE (estabilidade)

**Análise:**

São Paulo sendo um dos estados mais populosos está apresentando um maior número de casos.Esse dado sugere a necessidade de alocar mais recursos e reforçar as medidas de controle nessas áreas para conter a disseminação do vírus.

**5. Evolução Temporal dos Casos e Mortes**

O gráfico de linha mostra a evolução dos casos e mortes ao longo do tempo, comparando os dados diários com a média móvel de 7 dias.

**Análise:**

Podemos observar picos significativos durante certos períodos, correlacionando-se com relaxamentos nas medidas de distanciamento social. A reintrodução de restrições pode ser necessária durante esses picos para controlar a transmissão.

**Recomendações**
**Intensificar a Campanha de Vacinação:**

Aumentar a taxa de imunização completa, com foco especial na administração da 3ª dose.

**Reforçar Medidas de Controle:**

Implementar medidas mais rigorosas de distanciamento social, uso de máscaras e higiene das mãos.

**Alocar Recursos para Áreas Críticas:**

Direcionar esforços e recursos para os estados mais afetados, conforme indicado pela distribuição geográfica dos casos.

**Monitoramento Contínuo:**

Manter uma vigilância constante sobre a evolução dos casos e mortes para ajustar rapidamente as estratégias de intervenção.

Este dashboard nos fornece uma visão clara e abrangente do estado da pandemia no Brasil no ano de 2021, permitindo que tomemos decisões informadas para proteger a saúde pública e mitigar os impactos socioeconômicos.

Agradeço pela atenção e estou à disposição para quaisquer perguntas ou esclarecimentos adicionais.

Atenciosamente,

**Fabio Pereira**
**Analista de Dados**
**Vitória-ES **

**E-mail:fabioalves.js15@gmail.com**